# Docker Engineのインストール

---

構築したOpenHPC環境に[Docker Engine](https://www.docker.com/)をインストールします。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* OpenHPC環境の各ノードに対してAnsibleで操作できるように設定されている

VCノードを作成時に指定した値を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

各ノードに対してAnsibleによる操作が行えることを確認します。操作対象となる UnitGroup 名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

疎通確認を行います。

In [ ]:
!ansible {ugroup_name} -m ping

計算ノードのグループに対して疎通確認を行います。

In [ ]:
target = f'{ugroup_name}_compute'
!ansible {target} -m ping

## Docker Engineのインストール

[Install Docker Engine on CentOS](https://docs.docker.com/engine/install/centos/)の手順に従い Docekr Engineを計算ノードにインストールします。

Docker のレポジトリを追加します。

In [ ]:
!ansible {target} -b -m yum -a 'name=yum-utils'
!ansible {target} -b -a 'yum-config-manager --add-repo \
    https://download.docker.com/linux/centos/docker-ce.repo'

Docker Engine をインストールします。

In [ ]:
!ansible {target} -b -m yum -a 'name=docker-ce,docker-ce-cli,containerd.io'

Docker Engineのサービスを開始させます。

In [ ]:
!ansible {target} -b -m systemd -a 'name=docker enabled=yes state=started'

Docker Engineが利用できることを確認するために `docker info`コマンドを実行してみます。

In [ ]:
!ansible {target} -a 'docker info'

コンテナを実行できることを確認するために`hello-world`コンテナを実行してみます。

In [ ]:
!ansible {target} -a 'docker run --rm hello-world'

## NVIDIA Container Toolkit のインストール

[Setting up NVIDIA Container Toolkit](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html#id2)の手順に従いインストールを行います。

> 計算ノードでGPUが利用できる場合のみ NVIDIA Container Toolkitのインストールが必要となります。計算ノードでGPUが利用できない場合はこの節の実行は不要です。

レポジトリを追加します。

In [ ]:
!ansible {target} -b -m shell -a \
    'distribution=$(. /etc/os-release;echo rhel$VERSION_ID) \
    && curl -s -L https://nvidia.github.io/nvidia-docker/$distribution/nvidia-docker.repo | tee /etc/yum.repos.d/nvidia-docker.repo'

`nvidia-docker2`パッケージをインストールします。

In [ ]:
!ansible {target} -b -m dnf -a \
    'name=nvidia-docker2 update_cache=yes'

変更を反映するために Docker Engine を再起動します。

In [ ]:
!ansible {target} -b -a 'systemctl restart docker'

変更が反映されたことを確認するためにDockerの情報を表示させます。`Server`の `Runtimes`に`nvidia`が追加されたことを確認してください。
表示例を以下に示します。

```
Client:
 Context:    default
 Debug Mode: false
 Plugins:
  app: Docker App (Docker Inc., v0.9.1-beta3)
  buildx: Build with BuildKit (Docker Inc., v0.5.1-docker)
  scan: Docker Scan (Docker Inc.)

Server:
(中略)
 Runtimes: io.containerd.runc.v2 io.containerd.runtime.v1.linux nvidia runc
 Default Runtime: runc
(以降略) 
```

In [ ]:
!ansible {target} -b -a 'docker info'